In [1]:
#Setup
import pandas as pd
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">How can we create a Manual Event?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">What is the schema of a Manual Event?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">How can we use templating to enrich our notifications based on a subscription to a Manual Event?</p></li>
</ul>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Creating a Manual Event</p>

In [2]:
import pprint

import lusid_notifications
from lusidjam import RefreshingToken
from fbnsdkutilities import ApiClientFactory

api_factory_notifications = ApiClientFactory(lusid_notifications, token=RefreshingToken())
event_types_api = api_factory_notifications.build(lusid_notifications.api.EventTypesApi)
events_api = api_factory_notifications.build(lusid_notifications.api.EventsApi)
subscriptions_api = api_factory_notifications.build(lusid_notifications.api.SubscriptionsApi)
notifications_api = api_factory_notifications.build(lusid_notifications.api.NotificationsApi)

subscription_scope="FinbourneUniversity"
subscription_code="ManualEventSubscription"

In [3]:
try:
    subscriptions_api.get_subscription(
        scope=subscription_scope,
        code=subscription_code)
    
    subscriptions_api.delete_subscription(
        scope=subscription_scope,
        code=subscription_code
    )
    
except lusid_notifications.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise

In [4]:
subscriptions_api.create_subscription(
    create_subscription=lusid_notifications.CreateSubscription(
        id=lusid_notifications.ResourceId(
            scope=subscription_scope,
            code=subscription_code,
        ),
        display_name="New Trade",
        description="Listens for new trade executed Manual events",
        status="Active",
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type="Manual",
            filter="Subject eq 'Trade Executed'"
        )
    )
)

create_response = notifications_api.create_sms_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_sms_notification=lusid_notifications.CreateSmsNotification(
        description="New Trade Executed",
        body="""
                Trade {{jsonMessage.TradeId}} for {{jsonMessage.Quantity}} shares in 
                {{jsonMessage.Security}} executed.

                {{#jsonMessage.Tags}}
                - {{.}}
                {{/jsonMessage.Tags}}
            """,
        recipients=[
            "+447700900000",
            "+12334550000"
        ]
    )
)


In [5]:
event_types_api.get_event_type("Manual")


{'application': None,
 'description': 'Manual event',
 'entity': None,
 'event_name': 'Manual event',
 'id': 'Manual',
 'json_schema': {'properties': {'EventTime': {'format': 'date-time',
                                              'type': 'string'},
                                'EventType': {'type': 'string'},
                                'Id': {'maxLength': 64,
                                       'minLength': 1,
                                       'pattern': '^[a-zA-Z0-9\\-_]+$',
                                       'type': 'string'},
                                'JsonMessage': {},
                                'Message': {'maxLength': 2147483647,
                                            'minLength': 1,
                                            'pattern': '^[\\s\\S]*$',
                                            'type': 'string'},
                                'Subject': {'maxLength': 256,
                                            'minLength': 1,
      

In [6]:
events_api.create_event(
    body={
        "id": "TradeId123",
        "eventTime": "2023-02-10",
        "eventType": "Manual",
        "message": "A new trade has been executed",
        "subject": "Trade Executed",
        "jsonMessage": {
            "TradeId": "123",
            "Security": "Apple Ord Shares",
            "Quantity": 5100,
            "Tags": ["Buy", "Long", "ExchangeSettled"]
        }
    }
)


{'id': 'cf551917-3707-449a-9cd1-b92cf81998cd'}

In [7]:
events_api.create_event(
    body={
        "id": "TradeId123",
        "eventTime": "2023-02-10",
        "eventType": "Manual",
        "message": "A new trade has been executed",
        "subject": "Trade Executed",
        "jsonMessage": {
            "TradeId": "123",
            "Quantity": 5100
        }
    }
)


{'id': 'f681b513-34c6-4664-87d4-2279879e8259'}

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What have we covered?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We subscribed to a Manual Event instead of an automatically generated event.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We used templating to create a text message notification which used optional fields from the Manual Event.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We created a Manual Event which contained all the fields used in the template and saw that these become populated in the text message notification.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We created a Manual Event which was missing some of the fields used in the template and saw that these were left blank in the text message notification.</p></li>
</ul>

In [8]:
subscriptions_api.delete_subscription(
    scope=subscription_scope,
    code=subscription_code
)